# Intro to nlp
Natural language processing is field of machine learning in which we focus on language interaction of computer with that of human.it is subfield of linguistic,computer science,information engineering concerned with interation between computer and human language..Their are so many application of nlp.For example when you search on google and by mistake you type wrong spelling it automatically correct it and get you the most relevant results.Many chatting apps came with option of smart reply.In short,we encountered nlp algorithm in our day to day life.

# What this  competition is all about?
This competition is ideal for anyone who is getting started with nlp.This competiotion is all about making and evaluating a machine learning model that can predict whether a tweet is about disaster or not.
In this competion there were three datasets ,train,test and submission files.We will use train datset to train our machine learning algorithm.
Train contains following  columns:-
* text-The text of tweet
* keyword-tweet containg keyword
* Location-The location from where tweet was sent
* target-This is the columns which say whether a tweet if about disaster or not
* id-It just contains a uniques number for each row
Dropping the target columns test  data contains same above columns.Submission file is give just to take a look at the format in which submmission is required.

**Evaluation Metrics**

Every machine learning model is evaluated based on some some metrics.The evaluation metrics that is used in this competition is "F1 scoring".F1 is the reciprocal of precision and recall .A good f1 score means you have low false positive and low false negative.Keeping it simple  a good model has high f1 score close to 1 while a total failure of model to predict the true positive  brings the score to zero.


# Contents
* [Importing libraries](#imports)
* [Reading Dataset ](#reading)
* [Basic EDA](#eda)
* [Text processing](#processing)
* [Vectorization](#vectorization)
* [Classification Model](#models)

# Importing libraries  <a name="imports" a > 

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 
import pandas as pd
import numpy as np
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import cross_val_score
import re
from sklearn .metrics import classification_report
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
from nltk.tokenize import WhitespaceTokenizer,word_tokenize

# <a name="reading"></a> Reading Dataset

In [ ]:
#reading train and test data
train=pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test=pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [ ]:
print(f"training_data_shape:{train.shape}")
print(f"testing_data_shape:{test.shape}")

# <a name="eda"></a>Basic EDA

Take a look at the top ten rows

In [ ]:
train.head(10)

## Missing Values


In [ ]:
#function to get percentage of Nan in a Column
def get_nan(data):
    s=data.isna().sum()
    per=s/data.shape[0]
    return per*100

In [ ]:
train_nan=get_nan(train)
test_nan=get_nan(test)
nan_data=pd.DataFrame({"train":train_nan,"test":test_nan})
nan_data

So here we can see that  80 percent of keyword columns  in training data are Nan and 79 percent in test data. The test data doesn't contain target column so it shows Nan above.We will feed our model with only text columns so there's no need to  deal with Nan in other columns so i will leave them as it is.

In [ ]:
train.info()

## Know your target!! 

In [ ]:
sns.countplot(x='target',data=train,palette='Blues')


**Target columns contains two classes**
* 1 - if the tweet is about real disaster
* 0 - if the tweet is not about disaster 

## Keywords!!

In [ ]:
sns.barplot(y=train['keyword'].value_counts()[:20].index,x=train['keyword'].value_counts()[:20])

Above figure shows the top 20 keywords based on frequency of occurence.We can see that most of keywords are around 40. 

## Location Column

In [ ]:
train['location'].replace({'United States':'USA',
                           'New York':'USA',
                            "London":'UK',
                            "Los Angeles, CA":'USA',
                            "Washington, D.C.":'USA',
                            "California":'USA',
                             "Chicago, IL":'USA',
                             "Chicago":'USA',
                            "New York, NY":'USA',
                            "California, USA":'USA',
                            "FLorida":'USA',
                            "Nigeria":'Africa',
                            "Kenya":'Africa',
                            "Everywhere":'Worldwide',
                            "San Francisco":'USA',
                            "Florida":'USA',
                            "United Kingdom":'UK',
                            "Los Angeles":'USA',
                            "Toronto":'Canada',
                            "San Francisco, CA":'USA',
                            "NYC":'USA',
                            "Seattle":'USA',
                            "Earth":'Worldwide',
                            "Ireland":'UK',
                            "London, England":'UK',
                            "New York City":'USA',
                            "Texas":'USA',
                            "London, UK":'UK',
                            "Atlanta, GA":'USA',
                            "Mumbai":"India"},inplace=True)

sns.barplot(y=train['location'].value_counts()[:5].index,x=train['location'].value_counts()[:5],
            orient='h')

 Most of the tweets are from USA followed by Uk and so on.

# <a name="processing"></a>Text Processing
Getting data in required forma is  so that it can be feed into ml model is the most time taking and an important part of any data scince project.In nlp,their are several general steps to be followed for data cleaning.
* In this project first we will make all the words lowercase so that no two same words treated differently just beacause one in lowercase and then other in lowercase. 

* After that,i will remove the punctuation,unwanted strings like "yibf6,uds8mdh" which isn't a standard word .

* Just cleaning the data is not enough for any nlp pipeline,now we will move on to our next step to tokenize the data.

**Tokenization**:Tokens are the building block for ml model.Tokenization is the process of breaking the text into  smaller pieces  of strings called tokens.It can be of three types word,subword and subword.We will use word tokenization in our project.For example when we want to tokenize the sentence "He is running." considering space as delimiter we can break the sentence and form a token of words [He,is,running].
For more info <a href="https://www.analyticsvidhya.com/blog/2020/05/what-is-tokenization-nlp/" >visit</a>

* Now after cleaning the text and tokenizing it's time to remove stopwords from the data.Since they are frequently used in a sentence and didn't convey much information we should remove them.

In [ ]:
#function to clean text
def clean_text(text):
    text=text.lower()
    text = re.sub('\[.*?\]', "", text)
    text = re.sub('https?://\S+|www\.\S+', "", text)
    text=re.sub('<.*?>+','',text)
    text=re.sub(f"[{string.punctuation}]",'',text)
    text=re.sub('\n','',text)
    text=re.sub('\w\d\w','',text)
    return text
   
    

In [ ]:
#applying the cleaning process on both train and test
train['text']=train['text'].apply(lambda x:clean_text(x))
test['text']=train['text'].apply(lambda x:clean_text(x))
disaster_tweet=train[train['target']==1]['text']
non_disaster_tweet=train[train['target']==0]['text']

In [ ]:
from wordcloud import WordCloud
fig,(ax1,ax2)=plt.subplots(1,2,figsize=[28,7])
wordcloud1=WordCloud(background_color='white',height=100,width=200).generate(" ".join(disaster_tweet))
ax1.imshow(wordcloud1)
ax1.axis('off')
ax1.set_title('Disaster Tweets',fontsize=32)
wordcloud2=WordCloud(background_color='white',height=100,width=200).generate(" ".join(non_disaster_tweet))
ax2.imshow(wordcloud2)
ax2.axis('off')
ax2.set_title('Non-Disaster Tweets',fontsize=32)

In [ ]:
#function to remove stopwords
def stop_words_removal(text):
    text=[w for w in  text.split(" ")  if w not in stopwords.words('english')]
    return " ".join(text)
#applying the function on train and test
train['text']=train['text'].apply(lambda x: stop_words_removal(x))
test['text']=test['text'].apply(lambda x:stop_words_removal(x))

In [ ]:
#looking at top three rows after cleaning
train['text'][0:3]

# <a name="vectorization"></a>Vectorization
Computers didn't make sense of word as we humans do.They are good at crunching numbers.So,in a nlp pipeline we must convert  the words into numbers.But how do we can do it? One very general approach  is we will map words from a vocablury to vectors of real numbers.Here,vocablury means set of unique words included in text corpus.For example let's consider two sentence "kaggle is best." and "kaggle is good." .Now the vocablury consist of [kaggle,is,best,good].In vectors form when we write first sentence we can write it as [1,1,0,1] and similarly 2nd one as [1,1,1,0].Thsi is just tip of ice-berg we can do much more with word-vectors .To learn more here is  a nice <a href="https://medium.com/@jayeshbahire/introduction-to-word-vectors-ea1d4e4b84bf">blog</a>  

## Tf-idf
Tf-idf is method to penalize most occuring word in word vectors.This method rescale the frequency of words of how often it occur in a document.
* Term frequency
 Term frequency :It is scoring of frequency of words
 In short it refers to 
(Number of times word w occur in document)/(Total number of terms in document)
* Inverse Document Frequency
Inverse Document frequency :It measures the rarity of a word.
(1+log(N/n)) where N is total document and n is number of document in which word w occur

Tf-idf=term_frequency*inverse_document_frequency


In [ ]:
tf_idf=TfidfVectorizer(max_features=1000)
train_X=tf_idf.fit_transform(train['text'])
test_X=tf_idf.transform(test['text'])

# <a name="models"></a>Model Building
With everything set,it's time to make a working model for us to predict the class of tweets.


## Logistic Regression 

In [ ]:
from sklearn.linear_model import LogisticRegression
train_Y=train['target']
log_reg=LogisticRegression(C=2.02)
scores=cross_val_score(log_reg,train_X,train_Y,cv=5,scoring='f1')
print(scores)

We get a decent f1 score with logistic regression

In [ ]:
log_reg.fit(train_X,train_Y)

## let's try Naive Bayes 

In [ ]:
from sklearn.naive_bayes import MultinomialNB
Bayes=MultinomialNB(alpha=1.5)
scores_bayes=cross_val_score(Bayes,train_X,train_Y,cv=5,scoring='f1')
print(scores_bayes)

In [ ]:
Bayes.fit(train_X,train_Y)

## XgBoost 

In [ ]:
from   xgboost import XGBClassifier as xgb
xg=xgb(n_estimators=250,max_depth=12,colsample_bytree=0.1,learning_rate=0.2,subsample=0.4)
xg_scores=cross_val_score(xg,train_X,train_Y,cv=5,scoring='f1')
print(xg_scores)

In above three models naive bayes performs best we make our submission with it.

## Time to submit!! 

In [ ]:
def submit_model(path,model,vectors):
    sample_submission=pd.read_csv(path)
    sample_submission['target']=model.predict(vectors)
    sample_submission.to_csv('submission.csv',index=False)
submission_file_path = "../input/nlp-getting-started/sample_submission.csv"
test_vectors=test_X
submit_model(submission_file_path,Bayes,test_vectors)

## Final Thoughts 
So We now have a model that can predict whether a tweet is disasterous or not.If you find this kernel please upvote it.I take most of the help from this <a href="https://www.kaggle.com/parulpandey/getting-started-with-nlp-feature-vectors">kernel</a>.